# IMPORTAÇÃO DO MAILING DA ALELO

## PREPARAÇÃO DAS EXPRESSÕES REGULARES

In [1]:
import re

#: RECURSO AINDA NÃO UTILIZADO
LIST_CLEAR = ('NÆo Informado',)

MAP_LOGRADOURO = {
    'R': 'RUA', 
    'R.': 'RUA', 
    'EST': 'ESTRADA', 
    'AV': 'AVENIDA', 
    'AV.': 'AVENIDA', 
    'PC': 'PRAÇA', 
    'PRACA': 'PRAÇA', 
    'AL': 'ALAMEDA', 
    'LRG': 'LARGO'
}

MAP_PREFIXO_BAIRRO = {
    'JD': 'JARDIM', 
    'JD.': 'JARDIM', 
    'JRD': 'JARDIM',
    'JARD': 'JARDIM', 
    'PQ': 'PARQUE', 
    'PRQ': 'PARQUE',
    'VL': 'VILA', 
    'V': 'VILA', 
    'MRO': 'MORRO', 
    'STO': 'SANTO', 
    'STA': 'SANTA', 
    'IND': 'INDUSTRIAL', 
    'PRES': 'PRESIDENTE'
}

MAP_COMPLEMENTO = {
    'LJ': 'LOJA', 
    'SB LJ': 'SOBRE LOJA', 
    'SL': 'SALA',
    'FDS': 'FUNDOS'
}

STR_REGEX_PREFIXO_BAIRRO = '(JD|JD.|JARDIM|PQ|VL|V)'
STR_REGEX_LOGRADOURO = '(RUA|R\.?|EST|AVENIDA|AV\.?|PC|PRACA|AL|ALAMEDA|AVENIDA|LRG)'

REGEX_LOGRADOURO = re.compile(f'(?i)^{STR_REGEX_LOGRADOURO}')                                  
REGEX_PREFIXO_BAIRRO = re.compile(f'^({STR_REGEX_PREFIXO_BAIRRO}.*)')
REGEX_LETTERS = re.compile('^([A-Z ][A-Z0-9 \']+)$')
REGEX_REFERENCES = re.compile('^(ANTIGO[ ]\d+)(.*)$')


"""
ATUA NO CAMPO `COMPLEMENT` 
O OBJETIVO É IDENTIFICAR LETRAS ISOLADAS QUE POTENCIALMENTE FAZ PARTE DA NUMERAÇÃO
"""
REGEX_NUMBER_COMPLEMENT = re.compile("""^
                                        ([A-Z])     # Letra isolada
                                        [ ]         # Espaço obrigatório 
                                        (.*)        # Nome do bairro
                                        $""", flags=re.VERBOSE)

"""
ATUA NO CAMPO `FULL_NAME` QUE POSSUI O NOME DO LOGRADOURO
O OBJETIVO É QUEBRAR EM ATÉ 4 PARTES:
- TIPO
- NOME
- NÚMERO
- COMPLEMENTO
"""
REGEX_ADDRESS = re.compile(f"""(?i)                                                 # Ignorar caixa da letra
                               ^                                                    # Começo de linha
                               {STR_REGEX_LOGRADOURO}                               # Tipo de logradouro   - GRUPO 1
                               (.*?(?:[ ]DE[ ]\d{{4}})?)-?                          # Nome do logradouro   - GRUPO 2
                               (?:,[ ]?)?
                               (?:N[ ]?(?=\w)|0*)?
                               ((?:BOX[ ])?\d+(?:[ ]?[A-Za-z])?|S/N)?               # Número no logradouro - GRUPO 3
                               (?(3)([ ]+(?:(?:CS|LJ|SL|SALA|APT)[ ]?\d+)|\w+))?    # Complemento          - GRUPO 4
                               $""", flags=re.VERBOSE)        


"""
ATUA NO CAMPO `COMPLEMENT` QUE POSSUI O NOME DO BAIRRO
O OBJETIVO É IDENTIFICAR O NÚMERO 
"""
REGEX_START_NUMBER = re.compile(f"""^
                                    (?:N(?:R|:|.)?[ ]*)?
                                    ([\d/ ]+) # Número no logradouro - GRUPO 1
                                    (?:/)?    # Possível barra separadora
                                    (.*)      # Nome do Bairro - GRUPO 2
                                    $""", flags=re.VERBOSE)


"""
ATUA NO CAMPO `COMPLEMENT` QUE POSSUI O NOME DO BAIRRO
O OBJETIVO É IDENTIFICAR PARTES QUE CARACTERIZAM UM COMPLEMENTO DE ENDEREÇO
"""
REGEX_COMPLEMENT = re.compile("""^                             # Começo de linha
                                 [ ]*                          # Possíveis espaços 
                                 (
                                   TERREO|
                                   FUNDOS(?:[ ]SL[ ]?\d+)?|
                                   FDS|
                                   SB[ ]LJ|
                                   BL[ ]\w+|
                                   BL:\w[ ]APT:\w|
                                   (?:QUIOSQUE|KIOSQUE|BOX|BX|LJ|LOJA|SL|SALA|CS|CASA|PS)[ ]?[\w/,]+|
                                   (?:APT?|LT|LJ|LOJA|RUA|LOTE)[ ]\w+[ ](?:TORRE|QD|PS|COHAB|BOX|QUADRA)[ ]\w+
                                 )                             # Complementos possíveis - G1
                                 (?:[ ]?-?[ ]?)?               # Possível traço separador
                                 (.*)                          # Bairro - GRUPO 2
                                 $""", flags=re.VERBOSE)

"""
GERAÇÃO DE LOG PARA ANÁLISE
"""
def log(idx, target, done):
    if idx + 2 == -1:
        print(f'Linha: {idx} - Target: {target} - Done: {done}')

## HIGIENIZAÇÃO

Aqui geramos a string com o endereçamento final da EC.

In [2]:
import csv

#: ARQUIVO COM OS DADOS EXTRAÍDOS DO MAILING ORIGINAL
arquivo_csv_extracao = r'E:\FONTES_SOFIE\serverless_aws\LOTES\20190617_PRIMEIRO_LOTE_B_PREPARACAO.csv'

FINAL_LIST = list()   
    
with open(arquivo_csv_extracao, 'rt') as file_handler:
    csv_handler = csv.DictReader(file_handler, delimiter=';')
    FINAL_HEADERS = csv_handler.fieldnames.copy()
    
    for idx, row in enumerate(csv_handler):
#         #: Tratamento do CNPJ
#         target = row['CNPJ']
#         target = target.replace('.', '')
#         row['CNPJ'] = f'{target[:-12] if len(target[:-12]) == 2 else "0" + target[:-12]}.{target[-12:-9]}.{target[-9:-6]}/{target[-6:-2]}-{target[-2:]}'
        
#         #: Tratamendo do *zip_code*
#         target = row['zip_code']
#         if len(target) < 8:
#             target = '0' * (8 - len(target)) + target
#         row['zip_code'] = f'{target[:5]}-{target[5:]}'
        
        #: Tratamento do *full_name* 
        target = row['address.full_name'].strip().upper()
        
        if target.startswith('3 V'):
            target = target.replace('3 V', 'AV', 1)
        elif target.startswith('3'):
            target = target.replace('3', 'RUA', 1)
        elif not REGEX_LOGRADOURO.search(target):
            target = 'RUA ' + target
        
        parts = REGEX_ADDRESS.search(target)
        if parts:
            row['address.type'] = MAP_LOGRADOURO.get(parts[1].upper(), parts[1])
            row['address.full_name'] = parts[2].strip().upper()
            row['address.number'] = parts[3].strip().upper() if parts[3] else ''
            row['address.complement'] = ((parts[4].strip().upper() + ' ') if parts[4] else '') + row['address.complement']
        else:
            raise Exception(f'address.full_name: [{target}]')
        log(idx, target, parts is not None)
            
        #: Tratamento do *complement*
        original = row['address.complement']
        target = row['address.complement']
        
        if target in ('NAO SE APLICA', 'NAO INFORMADO'):
            row['address.complement'] = target = ''
        
        if target:
            target = target.upper()
            done = False
            
            #: REESCREVE ALGUNS PREFIXOS ABREVIADOS PARA A FORMA LITERAL
            for key, value in MAP_PREFIXO_BAIRRO.items():
                target = re.sub(f'(^|(?<=\s)){key}(?=\s)', value, target)              
                
            #: -------------------------------------------------------------------------------------
            
            #: CARACTERIZAÇÃO DE COMPLEMENTO
            if not done:
                parts = REGEX_REFERENCES.search(target)
                if parts:
                    row['address.reference'] = parts[1].strip().upper()
                    row['address.district'] = parts[2].strip().upper()
                    row['address.complement'] = ''
                    done = True
            log(idx, target, done)

            #: IDENTIFICANDO DESCRIÇÃO DE APTO, DESLOCANDO PARA COMPLEMENTO
            if not done:
                parts = REGEX_COMPLEMENT.search(target)
                if parts:
                    row['address.complement'] = parts[1].strip().upper()
                    row['address.district'] = parts[2].strip().upper()
                    done = True
            log(idx, target, done)
                    
            #: SEPARA NÚMERO DO LOGRADOURO COM O NOME DO BAIRRO
            if not done:
                parts = REGEX_START_NUMBER.search(target)
                if parts:
                    row['address.complement'] = ''
                    row['address.number'] += parts[1].strip().upper()
                    row['address.district'] = parts[2].strip().upper()
                    done = True
            log(idx, target, done)

            #: CONSIDERA COMO NOME DO BAIRRO DE ACORDO COMO ALGUNS PREFIXOS CONHECIDOS
            if not done:
                parts = REGEX_PREFIXO_BAIRRO.search(target)
                if parts:
                    row['address.complement'] = ''
                    row['address.district'] = parts[1].strip().upper()
                    done = True
            log(idx, target, done)

            #: SE SÓ TIVER LETRAS E ESPAÇOS, CONSIDERA-SE COMO BAIRRO
            if not done:
                parts = REGEX_LETTERS.search(target)
                if parts:
                    row['address.complement'] = ''
                    row['address.district'] = parts[1].strip().upper()
                    done = True
            log(idx, target, done)
           
            #: ----------------------------------------------------------------------------------------
               
            #: DESLOCA LETRAS ISOLADAS PARA COMPLEMENTAR O NÚMERO
            target = row['address.district']
            parts = REGEX_NUMBER_COMPLEMENT.search(target)
            if parts:
                row['address.number'] += parts[1] 
                row['address.district'] = parts[2].strip().upper()
                
            #: ------------------------------------------------------------------------------------------
            
            #: ELIMINA BAIRROS DUPLICADOS
            target = row['address.district']
            if target:
                part_1 = target[:len(target) // 2].strip()
                part_2 = target[len(target) // 2:].strip()
                if part_1 == part_2:
                    row['address.district'] = part_1
                    
            #: -------------------------------------------------------------------------------------------
            
            #: NORMALIZA PARTES DO COMPLEMENTO PARA A FORMA LITERAL
            target = row['address.complement']
            for key, value in MAP_COMPLEMENTO.items():
                target = re.sub(key, value, target)
            else:
                row['address.complement'] = target
                
        #: -----------------------------------------------------------------------------------------------
        #: HIGIENIZAÇÃO FINAL - DUPLOS ESPAÇOS

            
        FINAL_LIST.append(row)
            
            
print('** FIM DO PROCESSAMENTO **')

** FIM DO PROCESSAMENTO **


## GERAÇÃO DOS ARQUIVOS JSON COM O RESULTADO FINAL

In [3]:
from datetime import datetime
from json import dumps, loads
from pprint import pprint
from uuid import uuid4
import os

ORIGINAL_FIELDS = (
    'NU_SO_EC', 
    'CNPJ', 
    'RAZAO_SOCIAL', 
    'NOME_FANTASIA', 
    'ENDEREÇO', 
    'COMPLEMENTO_ENDERECO', 
    'CIDADE',
    'UF',
    'CEP',
    'DDD_01',
    'FONE_01',
    'FONE_02',
    'PROPRIETARIO_01',
    'MCC',
    'DESCRICAO_MCC',
    'VISITA SOLICITADA EM'
)

ADDRESS_PARTS = (
    'postal_code',
    'type',
    'full_name',
    'number',
    'complement',
    'reference',
    'district',
    'city',
    'state',
    'country'
)

COD_VR = 45
COD_VA = 46
COD_VAVR = 47

for idx, row in enumerate(FINAL_LIST):
    original_task_id = None
    file_name = os.path.join(os.getcwd(), 'FINAL_RESULT', f'row_{idx + 2:0>4}.json')
    
    if os.path.isfile(file_name):
        with open(file_name, 'rt') as file_handler:
            original_task_id = loads(file_handler.read()).get('task_id', None)
    
    data = {
        'task_id': original_task_id or str(uuid4()),
        'company': 'alelo',
        'last_movement': datetime.utcnow().isoformat(),
        'status': {
            'when': datetime.utcnow().isoformat(),
            'state': 'NEW',
            'status': 'WAITING',
            'lot_reference': '17/06/2019 - SOFIE',
            'row_line': idx + 2
        },
        'task': {
            'category': 'in_person',
            'type': 'inspection',
            'name': 'PAT',
            'reward': 10
        },
        'sofie_place': {
            'name': row['NOME_FANTASIA'],
            'responsible': row['PROPRIETARIO_01']
        },
        'original': {key.replace(' ', '_'): row.get(key, '') for key in ORIGINAL_FIELDS},
        'pre_preparation_address': {key: row.get(f'address.{key}') for key in ADDRESS_PARTS},
        'variables': {
            'produto': COD_VAVR
        }
    }
    
    with open(file_name, 'wt') as file_handler:
        file_handler.write(dumps(data, indent=4))
        
print('** PROCESSAMENTO EFETUADO COM SUCESSO **')

** PROCESSAMENTO EFETUADO COM SUCESSO **
